In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Easy

In [2]:
df = pd.read_csv('../data/singapore_airlines_reviews.csv')
df.head()

,published_date,published_platform,rating,type,text,title,helpful_votes
0,2024-03-12T14:41:14-04:00,Desktop,3,review,We used this airline to go from Singapore to L...,Ok,0
1,2024-03-11T19:39:13-04:00,Desktop,5,review,The service on Singapore Airlines Suites Class...,The service in Suites Class makes one feel lik...,0
2,2024-03-11T12:20:23-04:00,Desktop,1,review,"Booked, paid and received email confirmation f...",Don’t give them your money,0
3,2024-03-11T07:12:27-04:00,Desktop,5,review,"Best airline in the world, seats, food, servic...",Best Airline in the World,0
4,2024-03-10T05:34:18-04:00,Desktop,2,review,Premium Economy Seating on Singapore Airlines ...,Premium Economy Seating on Singapore Airlines ...,0


In [3]:
df = df.drop(columns=['type', 'published_date'])
df['published_platform'] = (df['published_platform'] == 'Desktop')
df = df.dropna()

In [4]:
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

In [5]:
df.head()

,published_platform,rating,text,title,helpful_votes
0,True,3,We used this airline to go from Singapore to L...,Ok,0
1,True,5,The service on Singapore Airlines Suites Class...,The service in Suites Class makes one feel lik...,0
2,True,1,"Booked, paid and received email confirmation f...",Don’t give them your money,0
3,True,5,"Best airline in the world, seats, food, servic...",Best Airline in the World,0
4,True,2,Premium Economy Seating on Singapore Airlines ...,Premium Economy Seating on Singapore Airlines ...,0


In [6]:
df['text_modified'] = df['title'] + ' ' + df['text']
df['text_modified'] = df['text_modified'].apply(preprocess)
df = df.dropna()
df.head()

,published_platform,rating,text,title,helpful_votes,text_modified
0,True,3,We used this airline to go from Singapore to L...,Ok,0,ok we used this airline to go from singapore t...
1,True,5,The service on Singapore Airlines Suites Class...,The service in Suites Class makes one feel lik...,0,the service in suites class makes one feel lik...
2,True,1,"Booked, paid and received email confirmation f...",Don’t give them your money,0,dont give them your money booked paid and rece...
3,True,5,"Best airline in the world, seats, food, servic...",Best Airline in the World,0,best airline in the world best airline in the ...
4,True,2,Premium Economy Seating on Singapore Airlines ...,Premium Economy Seating on Singapore Airlines ...,0,premium economy seating on singapore airlines ...


In [7]:
train, test = train_test_split(df)
bow = CountVectorizer()
train_x = bow.fit_transform(train['text_modified']).toarray()
test_x = bow.transform(test['text_modified']).toarray()
train_x.shape, test_x.shape

((7499, 20641), (2500, 20641))

In [8]:
# будем считать негативными отзывами те, у которых рейтинг <= 3
train_y = (train['rating'] > 3)
test_y = (test['rating'] > 3)

In [9]:
model = LogisticRegression()
model.fit(train_x, train_y)
predicted = model.predict(test_x)
f1_score(test_y, predicted)

/home/nixiiee/miniconda3/envs/classical-ml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9477020602218701